How many businesses were founded before 1000?
Wow! That's a lot of variation between countries. In one country, the oldest business was only founded in 1999. By contrast, the oldest business in the world was founded back in 578. That's pretty incredible that a business has survived for more than a millennium.

I wonder how many other businesses there are like that.

In [ ]:
%%sql

-- Get the count of rows in businesses where the founding year was before 1000
SELECT COUNT(business)
FROM businesses
WHERE year_founded < 999;

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (1, 1)
    except AssertionError:
        assert False, "The results should have a single column and a single row."
def test_colnames():
    try:
        assert results.columns.tolist() == ['count']
    except AssertionError:
        assert False, "The results should have a column named `'count`."
def test_count():
    try:
        assert last_output.DataFrame().loc[0, 'count'] == 6
    except AssertionError:
        assert False, "The count of businesses founded before 1000 is incorrect."

Which businesses were founded before 1000?

Having a count is all very well, but I'd like more detail. Which businesses have been around for more than a millennium?

In [ ]:
%%sql

-- Select all columns from businesses where the founding year was before 1000
-- Arrange the results from oldest to newest
SELECT *
FROM businesses
WHERE year_founded < 999
ORDER BY year_founded;

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (6, 4)
    except AssertionError:
        assert False, "The results should have four columns and six rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['business', 'year_founded', 'category_code', 'country_code']
    except AssertionError:
        assert False, "The results should have the four columns from the `businesses` table."
def test_where_year_founded_lt_1000():
    try:
        assert results.loc[:, 'year_founded'].max() < 1000
    except AssertionError:
        assert False, "The most recent year founded is not before 1000."
def test_ordered_by_year_founded():
    try:
        assert results.loc[:, 'year_founded'].is_monotonic
    except AssertionError:

Exploring the categories

Now we know that the oldest, continuously operating company in the world is called Kongō Gumi. But was does that company do? The category codes in the businesses table aren't very helpful: the descriptions of the categories are stored in the categories table.

This is a common problem: for data storage, it's better to keep different types of data in different tables, but for analysis, you want all the data in one place. To solve this, you'll have to join the two tables together.

In [ ]:
%%sql

-- Select business name, founding year, and country code from businesses; and category from categories
-- where the founding year was before 1000, arranged from oldest to newest

SELECT business, year_founded, b.country_code, category
FROM businesses AS b
INNER JOIN categories AS c
ON b.category_code = c.category_code
WHERE year_founded <999
ORDER BY year_founded;

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (6, 4)
    except AssertionError:
        assert False, "The results should have four columns and six rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['business', 'year_founded', 'country_code', 'category']
    except AssertionError:
        assert False, "The results should have business, year founded, and country code columns from the `businesses` table and category from the `categories` table."
def test_where_year_founded_lt_1000():
    try:
        assert results.loc[:, 'year_founded'].max() < 1000
    except AssertionError:
        assert False, "The most recent year founded is not before 1000."
def test_ordered_by_year_founded():
    try:
        assert results.loc[:, 'year_founded'].is_monotonic
    except AssertionError:
        assert False, "The rows are not ordered by increasing year founded."

Counting the categories

With that extra detail about the oldest businesses, we can see that Kongō Gumi is a construction company. In that list of six businesses, we also see a café, a winery, and a bar. The two companies recorded as "Manufacturing and Production" are both mints. That is, they produce currency.

I'm curious as to what other industries constitute the oldest companies around the world, and which industries are most common.

In [ ]:
%%sql

-- Select the category and count of category (as "n")
-- arranged by descending count, limited to 10 most common categories
SELECT c.category, COUNT (c.category) AS n
FROM businesses AS b
INNER JOIN categories AS c
    ON b.category_code = c.category_code
GROUP BY c.category
ORDER BY n DESC
LIMIT 10;

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (10, 2)
    except AssertionError:
        assert False, "The results should have two columns and ten rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['category', 'n']
    except AssertionError:
        assert False, "The results should have a category column and a count column named `'n'`."
def test_ordered_by_desc_n():
    try:
        assert results.loc[:, 'n'].is_monotonic_decreasing
    except AssertionError:
        assert False, "The rows are not ordered by descending count."
def test_count():
    try:
        assert results.loc[:, 'n'].values.tolist() == [37, 22, 19, 16, 15, 7, 6, 6, 6, 4]
    except AssertionError:
        assert False, "The category counts are not correct."

Oldest business by continent

It looks like "Banking & Finance" is the most popular category. Maybe that's where you should aim if you want to start a thousand-year business.

One thing we haven't looked at yet is where in the world these really old businesses are. To answer these questions, we'll need to join the businesses table to the countries table. Let's start by asking how old the oldest business is on each continent.

In [ ]:
%%sql

-- Select the oldest founding year (as "oldest") from businesses,
-- and continent from countries
-- for each continent, ordered from oldest to newest

SELECT MIN (b.year_founded) AS oldest, c.continent
FROM businesses AS b
     INNER JOIN countries AS c
        ON b.country_code = c.country_code
GROUP BY continent
ORDER BY oldest;

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (6, 2)
    except AssertionError:
        assert False, "The results should have two columns and six rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['oldest', 'continent']
    except AssertionError:
        assert False, "The results should have columns named oldest, and continent."
def test_ordered_by_min_year_founded():
    try:
        assert results.loc[:, 'oldest'].is_monotonic
    except AssertionError:
        assert False, "The rows are not ordered by year founded."
def test_count():
    try:
        assert results.loc[:, 'oldest'].values.tolist() == [578, 803, 1534, 1565, 1772, 1809]
    except AssertionError:
        assert False, "The year founded values are not correct."

Joining everything for further analysis

Interesting. There's a jump in time from the older businesses in Asia and Europe to the 16th Century oldest businesses in North and South America, then to the 18th and 19th Century oldest businesses in Africa and Oceania.

As mentioned earlier, when analyzing data it's often really helpful to have all the tables you want access to joined together into a single set of results that can be analyzed further. Here, that means we need to join all three tables.

In [ ]:
%%sql

-- Select the business, founding year, category, country, and continent
SELECT b.business, b.year_founded, c1.category, c2.country, c2.continent
FROM businesses AS b
INNER JOIN categories AS c1
    ON b.category_code = c1.category_code
INNER JOIN countries AS c2
    ON b.country_code = c2.country_code

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (163, 5)
    except AssertionError:
        assert False, "The results should have five columns and one hundred and sixty three rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['business', 'year_founded', 'category', 'country', 'continent']
    except AssertionError:
        assert False, "The results should have columns named business, year_founded, category, country, and continent."

Counting categories by continent

Having businesses joined to categories and countries together means we can ask questions about both these things together. For example, which are the most common categories for the oldest businesses on each continent?

In [ ]:
%%sql

-- Count the number of businesses in each continent and category

SELECT c2.continent, c1.category, COUNT(*) AS n
FROM businesses AS b
INNER JOIN categories AS c1
    ON b.category_code = c1.category_code
INNER JOIN countries AS c2
    ON b.country_code = c2.country_code
GROUP BY c2.continent, c1.category
ORDER BY c2.continent

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (56, 3)
    except AssertionError:
        assert False, "The results should have three columns and fifty six rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['continent', 'category', 'n']
    except AssertionError:
        assert False, "The results should have continent, category, and count (as 'n')."
def test_count():
    try:
        assert results.loc[:, 'n'].sort_values(ascending=False).values.tolist() == [17, 12, 10, 9, 8, 7, 6, 5, 5, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    except AssertionError:
        assert False, "The counts are not correct."

Filtering counts by continent and category

Combining continent and business category led to a lot of results. It's difficult to see what is important. To trim this down to a manageable size, let's restrict the results to only continent/category pairs with a high count.

In [ ]:
%%sql

-- Repeat that previous query, filtering for results having a count greater than 5

SELECT c2.continent, c1.category, COUNT(*) AS n
FROM businesses AS b
INNER JOIN categories AS c1
    ON b.category_code = c1.category_code
INNER JOIN countries AS c2
    ON b.country_code = c2.country_code
GROUP BY c2.continent, c1.category
HAVING COUNT(*) > 5
ORDER BY n DESC

In [ ]:
%%nose
last_output = _

def test_resultset():
    try:
        assert str(type(last_output)) == "<class 'sql.run.ResultSet'>"
    except AssertionError:
        assert False, "Please ensure a SQL ResultSet is the output of the code cell."
results = last_output.DataFrame()
def test_shape():
    try:
        assert results.shape == (7, 3)
    except AssertionError:
        assert False, "The results should have three columns and seven rows."
def test_colnames():
    try:
        assert results.columns.tolist() == ['continent', 'category', 'n']
    except AssertionError:
        assert False, "The results should have continent, category, and count (as 'n')."
def test_count():
    try:
        assert results.loc[:, 'n'].values.tolist() == [17, 12, 10, 9, 8, 7, 6]
    except AssertionError:
        assert False, "The counts are not correct."